In [1]:
import numpy as np
import chemical_potential
import vapour_pressure as vp
import Bentheimer_Sandstone

### 水分状態を換算するクラス  
換算する前の水分状態と換算後の水分状態を指定し、温度と水分状態を与えることで値を換算するクラス  
$T$：絶対温度[K]  
$R_v$：水蒸気の気体定数  
$RH$：相対湿度[-]  
$P_v$：水蒸気圧[Pa]  
$P_{vs}$：飽和水蒸気圧[Pa]  
$\mu$：水分化学ポテンシャル[J/kg]  

RH（相対湿度） - Pv（水蒸気圧）　関係  
$RH = \frac{P_v}{P_{vs}}$  

RH（相対湿度） - Miu（水分化学ポテンシャル）　関係  
$\mu = R_v T log(RH)$

In [2]:
### 水分状態を換算するクラス ######################
class MoistureConversion():
    Rv = 8316.0/18.016
    def __init__(self, moistureIndexBefore, moistureIndexAfter):
        self.B = moistureIndexBefore
        self.A = moistureIndexAfter
        
    def convert(self, tem, moisture):
#   RH（相対湿度） - Pv（水蒸気圧）　関係
        if self.B == 'RH' and self.A =='Pv':
            pv = moisture* vp.Pvs(tem) 
            return pv
        elif self.B == 'Pv' and self.A =='RH':
            rh = moisture/ vp.Pvs(tem) 
            return rh
#   RH（相対湿度） - Miu（水分化学ポテンシャル）　関係
        elif self.B == 'RH' and self.A =='Miu':
            miu = self.Rv * tem* np.log(moisture) 
            return miu
        elif self.B == 'Miu' and self.A =='RH':
            rh = np.exp(moisture/(self.Rv*tem))
            return rh
#   Pv（水蒸気圧） - Miu（水分化学ポテンシャル）　関係
        elif self.B == 'Pv' and self.A =='Miu':
            miu = self.Rv * tem* np.log(moisture/vp.Pvs(tem)) 
            return miu
        elif self.B == 'Miu' and self.A =='Pv':
            pv = vp.Pvs(tem)*np.exp(moisture/(self.Rv*tem))
            return pv
#   換算が要らない場合
        elif self.B == self.A or self.B =='Nothing':
            return moisture
        else : 
            raise ValueError('各物性クラスのmoistureIndexには、「RH」か「Pv」,「Miu」を指定しください。')
################################################

In [3]:
# 使用例
c = MoistureConversion('Miu','RH')
c.convert(293.15, -1500.0)

0.988975971012088

### 物性値を取得するクラス  
各物性値ファイルに存在する物性値を元の形式のまま取得してくるクラス。  

#### 実験から熱水分物性値として考えられるもの  
【参考】建築材料の熱・空気・湿気物性値，日本建築学会，丸善株式会社，2001など

#### 熱物性値  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\rho_0$：密度[kg/m3]  
$c_0$：比熱[J/kg K]  
$c_0\rho_0$：容積熱容量[J/m3 K]  

$\lambda$：熱伝導率[W/mK]  
定義：$\dot q = -\lambda・grad\space T$  

$R$：熱抵抗[Km2/W]  

#### 水分物性値  
$\dot m_v$：単位時間当たりの水蒸気流量の密度[kg/m2 s]  
$\dot m_m$：単位時間当たりの水分流量の密度[kg/m2 s]  
$\phi$：含水率[m3/m3]  
他標記として$w$[kg/m3]、$u$[kg/kg]など  
$S$：飽和度[-]  
#### 気相水分伝導率  
$\delta_p$：透湿率[kg/m Pa s]　文中記号DP  
定義：$m_v = -\delta_p・grad\space p_v$  

$Z_p$：湿気抵抗[m2Pa s/kg]  
$\delta_v$：水蒸気の拡散係数[m2/s]  
定義：$\dot m_v = -\delta_v・grad\space \rho_v$

$k_T$：温度勾配水分伝導率[kg/m K s]  
定義：$\dot m_m = -k_T・grad\space T$  

$D_T$：温度勾配水分拡散係数[m2/K s]  
定義：$\dot m_m = -\rho_0 D_T・grad\space T$  

$\lambda^{'}_{\mu g}$：水分化学ポテンシャル勾配に対する気相水分伝導率[kg/ms (J/kg)]  
定義：$\dot m_v = -\lambda^{'}_{\mu g}・grad\space \mu$

$\lambda^{'}_{Tg}$：温度勾配に対する気相水分伝導率[kg/ms K]  
定義：$\dot m_v = -\lambda^{'}_{Tg}・grad\space \mu$

#### 液相水分伝導率  
$k_m$：水分伝導率[kg/m Pa s]  
定義：$\dot m_m = -k_m・grad\space s$

$D_w$：水分拡散係数[m2/s]  
定義：$\dot m_m = -\rho_0 D_w・grad\space u$  

$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に対する液相水分伝導率[kg/ms (J/kg)]  
定義：$\dot m_v = -\lambda^{'}_{\mu l}・grad\space \mu$

#### 空気の拡散係数  
$k_a$：透気率[m2/Ks]  


※新しい基準で物性値を測定した場合、本クラスに書き加えること


In [4]:
### 物性値ファイルに何が存在するかを確認するクラス
class GetMaterial():

    # 新しく物性モジュールを作成した場合は、ここに追加してください。
    mlist = {
        '合板' : 'Plywood', #物性値ファイルの名前
        'グラスウール' : 'GlassWool',
        'BentheimerSandstone': Bentheimer_Sandstone.Property()
    }
    
    def __init__(self, material,mainMoistureIndex):
        self.material = self.mlist[material]
        self.list = self.material.proplist
        self.mainMI = mainMoistureIndex

In [5]:
# 使用例
stone = GetMaterial('BentheimerSandstone','Miu')
stone.material.proplist
#print(stone.list)

{'DP': 'Miu',
 'DPhi': 'Miu',
 'Dw': 'Miu',
 'LAM': 'Miu',
 'Miu': 'Phi',
 'Phi': 'Miu',
 'crow': 'Miu'}

### 取得した物性値を流量計算に適した形に換算するクラス

In [9]:
class PropertyConversion(GetMaterial):
    
    Rv = 8316.0/18.016
    row = 1000.0
    grav = 9.806650
    
##########################################################
#   熱容量
    def crow(self, tem, moisture):
        if 'crow' in self.list:
            self.m = MoistureConversion(self.mainMI, self.list['crow']).convert(tem, moisture)
            return self.material.crow( tem, self.m )
        else : 
            raise ValueError('この材料ファイルは熱容量の情報「crow」を持っていません')
#   熱伝導率
    def LAM(self, tem, moisture ):
        if 'LAM' in self.list:
            self.m = MoistureConversion(self.mainMI, self.list['LAM']).convert(tem, moisture)
            return self.material.LAM( tem, self.m)
        else : 
            raise ValueError('この材料ファイルは熱伝導率の情報「LAM」を持っていません')
    
##########################################################
#   水分容量（気相の水分状態（ポテンシャル・相対湿度など）から含水率への変換）
    def Phi(self, tem, moisture):
        if 'Phi' in self.list:
            self.m = MoistureConversion(self.mainMI, self.list['Phi']).convert(tem, moisture)
            return self.material.Phi( tem, self.m ) 
        else : 
            raise ValueError('この材料ファイルは平衡含水率の情報「Phi」を持っていません')
#   含水率から水分化学ポテンシャルへの変換　#相対湿度・水蒸気圧などへの換算も増やそう
    def Miu(self, tem, phi):
        if 'Miu' in self.list:
            return self.material.moisture( tem, phi )
        else : 
            raise ValueError('この材料ファイルは含水率-水分化学ポテンシャル関係の情報「Miu」を持っていません')
#   含水率の水分化学ポテンシャル微分
    def DPhi(self, tem,moisture):
        return self.material.DPhi(tem, self.MC.convert(self, tem, moisture))
        
###     気相水分伝導率    #######
#   水蒸気圧勾配に対する気相水分伝導率（透湿率）
    def DP(self, tem, moisture):
        if 'DP' in self.list:
            self.m = MoistureConversion(self.mainMI, self.list['DP']).convert(tem, moisture)
            return self.material.LDMG( tem, self.m )
        else : 
            raise ValueError('この材料ファイルは該当する気相水分伝導率の情報「DP」を持っていません')

#   水分化学ポテンシャル勾配に対する気相水分伝導率
    def LDMG(self, tem, moisture):
        if 'LDMG' in self.list:
            self.m = MoistureConversion(self.mainMI, self.list['LDMG']).convert(tem, moisture)
            return self.material.LDMG( tem, self.m )
        elif 'DP' in self.list:
            self.m = MoistureConversion(self.mainMI, self.list['DP']).convert(tem, moisture)
            self.Miu= MoistureConversion(self.mainMI, 'Miu').convert(tem, moisture)
            DPDM = vp.Pvs(tem) *np.exp(self.Miu/self.Rv/tem) /self.Rv/tem
            return DPDM *self.material.DP( tem, self.m )
        else : 
            raise ValueError('この材料ファイルは該当する気相水分伝導率の情報「LDMG」、「DP」のいずれも持っていません')
        
#   温度勾配に対する気相水分伝導率
    def LDTG(self, tem, moisture):
        if 'LDTG' in self.list:
            self.m = MoistureConversion(self.mainMI, self.list['LDTG']).convert(tem, moisture)
            return self.material.LDTG( tem, self.m )
        elif 'DP' in self.list:    
            self.m = MoistureConversion(self.mainMI, self.list['DP']).convert(tem, moisture)
            self.Miu= MoistureConversion(self.mainMI, 'Miu')
            DPDT= np.exp(self.Miu/self.Rv/tem)*(vp.DPvs(tem)-vp.Pvs(tem)*self.Miu/self.Rv/(tem**2))
            return DPDT *self.material.DP( tem, self.m )
        else : 
            raise ValueError('この材料ファイルは該当する気相水分伝導率の情報「LDMT」、「DP」のいずれも持っていません')
    
###     液相水分伝導率    #######
#   水分化学ポテンシャル勾配に対する液相水分伝導率
    def LDML(self, tem, moisture):
        if 'LDML' in self.list:
            return self.material.LDML( tem, MoistureConversion(self.mainMI, self.list['LDML']).convert(tem, moisture) )
        elif 'Dw' in self.list:
            self.m = MoistureConversion(self.mainMI, self.list['Dw']).convert(tem, moisture)
            return self.row / self.grav * self.material.Dw( tem, self.m ) 
        else : 
            raise ValueError('この材料ファイルは該当する液相水分伝導率の情報「LDML」、「Dw」のいずれも持っていません')

################################################      


In [11]:
PropertyConversion('BentheimerSandstone','Miu').LDML(300.0,-100)
#p.crow(293.15, -100)

1.447770856656244e-10